# 4. VADER를 이용한 X(Twitter) 트윗 감정분석

### 4-1. 필수 라이브러리 Import
감정분석과 데이터 처리에 필요한 라이브러리들을 가져옵니다.

In [16]:
# 데이터 분석 및 조작을 위한 pandas 라이브러리
import pandas as pd

# VADER 감정분석 모델을 위한 NLTK 라이브러리
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# 정규표현식을 사용한 텍스트 전처리를 위한 re 라이브러리
import re

### 4-2. VADER 감정분석 사전 다운로드
VADER 감정분석에 필요한 어휘 사전을 다운로드합니다. (최초 1회만 실행)


In [28]:
# VADER 감정분석에 필요한 어휘 사전을 다운로드
# vader_lexicon: 감정 점수가 매핑된 단어 사전 데이터
# 최초 한 번만 실행하면 로컬에 저장됨
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\82102\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

### 4-3. 다중 사용자 트윗 데이터 병합
X_data 디렉토리의 모든 `user_*.csv` 파일들을 하나로 병합하여 통합 데이터셋을 생성합니다.


In [29]:
# 파일 시스템 작업에 필요한 추가 라이브러리 import
import glob  # 파일 패턴 매칭을 위한 라이브러리
import os    # 운영체제 인터페이스를 위한 라이브러리
from datetime import datetime  # 날짜/시간 처리를 위한 라이브러리

# glob 패턴을 사용하여 현재 디렉토리에서 "user_"로 시작하고 ".csv"로 끝나는 모든 파일 찾기
csv_files = glob.glob("user_*.csv")
print(f"발견된 모든 CSV 파일: {csv_files}")
print(f"총 {len(csv_files)}개의 파일을 병합합니다.")

# 각 CSV 파일의 데이터프레임을 저장할 리스트 초기화
dfs = []

# 발견된 모든 CSV 파일을 순회하며 데이터 로드
for file in csv_files:
    # 파일이 실제로 존재하는지 확인
    if os.path.exists(file):
        # CSV 파일을 pandas 데이터프레임으로 읽기
        temp_df = pd.read_csv(file)
        
        # 파일명에서 사용자명 추출 (예: "user_@elonmusk_tweets.csv" → "@elonmusk")
        username = file.replace("user_", "").replace("_tweets.csv", "")
        
        # 각 행에 해당 사용자명을 컬럼으로 추가
        temp_df['username'] = username
        
        # 리스트에 데이터프레임 추가
        dfs.append(temp_df)
        print(f"{file} 읽기 완료 - {len(temp_df)}개 행")

# 모든 데이터프레임을 하나로 병합
if dfs:
    # concat 함수를 사용하여 세로로 병합하고 인덱스 리셋
    df = pd.concat(dfs, ignore_index=True)
    print(f"🎉 총 {len(df)}개 행이 병합되었습니다!")
    print(f"📊 포함된 사용자 수: {len(df['username'].unique())}")
else:
    print("읽을 수 있는 파일이 없습니다.")

발견된 모든 CSV 파일: ['user_@Ajay_Bagga_tweets.csv', 'user_@BillAckman_tweets.csv', 'user_@CathieDWood_tweets.csv', 'user_@elonmusk_tweets.csv', 'user_@JDVance_tweets.csv', 'user_@LizAnnSonders_tweets.csv', 'user_@marcorubio_tweets.csv', 'user_@michaelbatnick_tweets.csv', 'user_@RayDalio_tweets.csv', 'user_@SecScottBessent_tweets.csv', 'user_@sundarpichai_tweets.csv', 'user_@tim_cook_tweets.csv', 'user_@WhiteHouse_tweets.csv']
총 13개의 파일을 병합합니다.
user_@Ajay_Bagga_tweets.csv 읽기 완료 - 530개 행
user_@BillAckman_tweets.csv 읽기 완료 - 800개 행
user_@CathieDWood_tweets.csv 읽기 완료 - 670개 행
user_@elonmusk_tweets.csv 읽기 완료 - 764개 행
user_@JDVance_tweets.csv 읽기 완료 - 701개 행
user_@LizAnnSonders_tweets.csv 읽기 완료 - 673개 행
user_@marcorubio_tweets.csv 읽기 완료 - 806개 행
user_@michaelbatnick_tweets.csv 읽기 완료 - 760개 행
user_@RayDalio_tweets.csv 읽기 완료 - 725개 행
user_@SecScottBessent_tweets.csv 읽기 완료 - 255개 행
user_@sundarpichai_tweets.csv 읽기 완료 - 585개 행
user_@tim_cook_tweets.csv 읽기 완료 - 838개 행
user_@WhiteHouse_tweets.csv 읽기 완료 -

### 4-4. URL 제거 함수 정의
트윗 텍스트에서 URL을 제거하는 전처리 함수를 정의합니다. URL은 감정분석에 불필요한 노이즈입니다.


In [30]:
def remove_urls(text):
    """
    텍스트에서 URL을 제거하는 함수
    
    Args:
        text (str): 원본 텍스트
        
    Returns:
        str: URL이 제거된 텍스트
    """
    # 정규표현식을 사용하여 HTTP/HTTPS URL 패턴을 찾아서 공백으로 대체
    # r'https?://\S+' 패턴 설명:
    # - https? : http 또는 https (? 는 s가 있거나 없거나)
    # - :// : 프로토콜 구분자
    # - \S+ : 공백이 아닌 문자가 1개 이상 연속 (URL의 나머지 부분)
    cleaned_text = re.sub(r'https?://\S+', '', text)
    
    # 앞뒤 공백 제거하여 반환
    return cleaned_text.strip()

### 4-5. 트윗 텍스트에서 URL 제거 적용
정의한 URL 제거 함수를 모든 트윗 텍스트에 적용하여 데이터를 정제합니다.


In [31]:
# pandas의 apply() 함수를 사용하여 모든 트윗 텍스트에 URL 제거 함수 적용
# apply()는 시리즈의 각 요소에 함수를 적용하여 새로운 시리즈를 반환
df['full_text'] = df['full_text'].apply(remove_urls)

### 4-6. 빈 문자열을 NaN으로 변환
URL 제거 후 빈 문자열이 된 텍스트들을 pandas의 NaN(결측값)으로 변환합니다.


In [32]:
# 빈 문자열('')을 pandas의 NA(Not Available) 값으로 변환
# replace() 함수: 첫 번째 인자를 두 번째 인자로 대체
# inplace=True: 원본 데이터프레임을 직접 수정 (새로운 객체 생성 안 함)
# pd.NA: pandas 2.0+에서 권장하는 결측값 표현
df['full_text'].replace('', pd.NA, inplace=True)

C:\Users\82102\AppData\Local\Temp\ipykernel_36128\1257741985.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['full_text'].replace('', pd.NA, inplace=True)


### 4-7. 결측값(NaN) 제거
감정분석이 불가능한 빈 텍스트나 결측값을 가진 행들을 데이터셋에서 제거합니다.


In [33]:
# dropna() 함수를 사용하여 결측값을 가진 행들을 제거
# subset=['full_text']: full_text 컬럼에서 NaN 값을 가진 행들만 제거
# inplace=True: 원본 데이터프레임을 직접 수정
# 감정분석을 위해서는 텍스트 내용이 필수이므로 빈 텍스트는 제거 필요
df.dropna(subset=['full_text'], inplace=True)

### 4-8. 날짜 형식 변환 및 데이터 정렬
트위터 API 형식의 날짜를 표준 날짜 형식으로 변환하고, 최신 순으로 정렬합니다.


In [34]:
def convert_date_format(date_str):

    try:
        # 트위터 API 날짜 형식: "Mon Jun 16 02:50:54 +0000 2025"
        # pd.to_datetime으로 파싱하고 strftime으로 원하는 형식으로 변환
        # format 매개변수 설명:
        # %a: 축약된 요일명 (Mon, Tue, ...)
        # %b: 축약된 월명 (Jan, Feb, ...)  
        # %d: 일 (01-31)
        # %H:%M:%S: 시:분:초
        # %z: 타임존 오프셋 (+0000)
        # %Y: 4자리 연도
        dt = pd.to_datetime(date_str, format='%a %b %d %H:%M:%S %z %Y')
        
        # 타임존 제거하고 "YYYY-MM-DD HH:MM:SS" 형식으로 변환
        return dt.strftime('%Y-%m-%d %H:%M:%S')
    except:
        # 파싱 실패 시 원본 그대로 반환
        return date_str

# 날짜 형식 변환 프로세스 시작
print("날짜 형식 변환 중...")

# 모든 트윗의 created_at 컬럼에 날짜 변환 함수 적용
df['created_at'] = df['created_at'].apply(convert_date_format)

# 문자열을 pandas datetime 객체로 변환 (정렬 및 시간 연산을 위해)
df['created_at'] = pd.to_datetime(df['created_at'])

# 최신 트윗이 맨 위에 오도록 내림차순 정렬
# ascending=False: 내림차순 (큰 값부터 작은 값 순)
# reset_index(drop=True): 정렬 후 인덱스를 0부터 다시 시작
df = df.sort_values(by='created_at', ascending=False).reset_index(drop=True)

print("날짜 형식 변환 및 정렬 완료!")
print(f"날짜 범위: {df['created_at'].min()} ~ {df['created_at'].max()}")
print("\n결과 확인:")
print(df.head())

날짜 형식 변환 중...
날짜 형식 변환 및 정렬 완료!
날짜 범위: 2020-01-17 01:05:39 ~ 2025-06-16 04:36:37

결과 확인:
           created_at                                          full_text  \
0 2025-06-16 04:36:37  1. Iran produces around 3.3mn barrels per day ...   
1 2025-06-16 03:40:49             Today at Apple. #F1TheMovie #Severance   
2 2025-06-16 03:04:38  Two countries, separated by 700 kms from each ...   
3 2025-06-16 03:01:08  BREAKING: Iranian opposition Telegram channels...   
4 2025-06-16 02:50:54  26 now.   Note the swing due east at the edge ...   

      username  
0  @Ajay_Bagga  
1    @tim_cook  
2  @Ajay_Bagga  
3  @BillAckman  
4  @BillAckman  


### 4-9. VADER 감정분석기 초기화
VADER(Valence Aware Dictionary and sEntiment Reasoner) 감정분석 모델을 초기화합니다.


In [35]:
# VADER 감정분석기 인스턴스 생성
# SentimentIntensityAnalyzer: VADER 알고리즘을 구현한 클래스
# - 소셜 미디어 텍스트에 특화된 감정분석 도구
# - 이모티콘, 대문자, 구두점, 단어 조합 등을 고려하여 감정 점수 계산
# - positive, negative, neutral, compound 점수를 반환
sia = SentimentIntensityAnalyzer()

### 4-10. 감정분석 함수 정의
텍스트를 분석하여 positive/negative/neutral로 분류하고 각 감정 점수를 반환하는 함수를 정의합니다.


In [36]:
def analyze_sentiment(text):
    # VADER 감정분석기로 텍스트 분석
    # polarity_scores() 반환값:
    # - 'neg': negative 감정 점수 (0~1)
    # - 'neu': neutral 감정 점수 (0~1)  
    # - 'pos': positive 감정 점수 (0~1)
    # - 'compound': 복합 점수 (-1~1, 전체적인 감정 강도)
    scores = sia.polarity_scores(text)
    
    # compound 점수를 기준으로 감정 분류
    compound = scores['compound']
    
    # VADER 권장 임계값을 사용한 감정 분류
    if compound >= 0.05:
        # compound >= 0.05: 긍정적 감정
        sentiment = 'positive'
    elif compound <= -0.05:
        # compound <= -0.05: 부정적 감정
        sentiment = 'negative'
    else:
        # -0.05 < compound < 0.05: 중립적 감정
        sentiment = 'neutral'
    
    # pandas Series로 반환 (DataFrame의 새 컬럼들로 할당하기 위함)
    return pd.Series([sentiment, scores['neg'], scores['neu'], scores['pos']])

### 4-11. 모든 트윗에 감정분석 실행
정의한 감정분석 함수를 모든 트윗 텍스트에 적용하여 감정 컬럼들을 생성합니다.


In [37]:
# pandas의 apply() 함수를 사용하여 모든 트윗에 감정분석 함수 적용
# analyze_sentiment() 함수가 pd.Series를 반환하므로 
# 여러 컬럼에 동시에 할당 가능
# - sentiment: 감정 분류 라벨 ('positive', 'negative', 'neutral')
# - neg: 부정 감정 점수 (0~1)
# - neu: 중립 감정 점수 (0~1)
# - pos: 긍정 감정 점수 (0~1)
df[['sentiment', 'neg', 'neu', 'pos']] = df['full_text'].apply(analyze_sentiment)

### 4-12. 최종 데이터 컬럼 선택
분석에 필요한 핵심 컬럼들만 선택하여 최종 데이터셋을 구성합니다.


In [38]:
# 최종 분석 결과에 필요한 컬럼들만 선택
# - created_at: 트윗 작성 날짜/시간
# - full_text: 전처리된 트윗 텍스트 (URL 제거됨)
# - username: 트윗 작성자 (@사용자명)
# - sentiment: 감정 분류 결과 (positive/negative/neutral)
# - neg: 부정 감정 점수
# - neu: 중립 감정 점수  
# - pos: 긍정 감정 점수
df = df[['created_at', 'full_text', 'username', 'sentiment', 'neg', 'neu', 'pos']]

### 4-13. 최종 결과 저장 및 요약
감정분석이 완료된 트윗 데이터를 CSV 파일로 저장하고 처리 결과를 요약합니다.


In [40]:
# 결과 파일 저장
output_filename = "merged_tweets_with_sentiment.csv"

# 감정분석이 완료된 데이터프레임을 CSV 파일로 저장
# index=False: 행 인덱스를 파일에 포함하지 않음
df.to_csv(output_filename, index=False)

# 처리 결과 요약 정보 출력
print(f"✅ 결과가 {output_filename}에 저장되었습니다.")
print(f"📊 총 {len(df)}개의 트윗이 처리되었습니다.")

# 감정분석에 포함된 사용자 목록 (알파벳 순 정렬)
print(f"👥 포함된 사용자: {sorted(df['username'].unique())}")

# 트윗 데이터의 시간 범위
print(f"📅 날짜 범위: {df['created_at'].min()} ~ {df['created_at'].max()}")

# 감정 분류별 트윗 수 통계
print(f"\n📈 감정 분류별 통계:")
sentiment_counts = df['sentiment'].value_counts()
for sentiment, count in sentiment_counts.items():
    percentage = (count / len(df)) * 100
    print(f"  - {sentiment}: {count}개 ({percentage:.1f}%)")

# 사용자별 트윗 수 통계 (상위 5명)
print(f"\n🏆 사용자별 트윗 수 (상위 5명):")
user_counts = df['username'].value_counts().head(5)
for username, count in user_counts.items():
    print(f"  - {username}: {count}개")

결과가 merged_tweets_with_sentiment.csv에 저장되었습니다.
총 8594개의 트윗이 처리되었습니다.
포함된 사용자: ['@Ajay_Bagga', '@BillAckman', '@CathieDWood', '@JDVance', '@LizAnnSonders', '@RayDalio', '@SecScottBessent', '@WhiteHouse', '@elonmusk', '@marcorubio', '@michaelbatnick', '@sundarpichai', '@tim_cook']
날짜 범위: 2020-01-17 01:05:39 ~ 2025-06-16 04:36:37
